# Collaborative filtering recommender system

## Dataset description

In this assignment you will use Apache Spark to build simple movie recommendation system on big MovieLens 20M dataset (https://grouplens.org/datasets/movielens/). The dataset contains 20 million ratings for 27000 movies given by 138000 users. 

Dataset format: comma-separated values (https://en.wikipedia.org/wiki/CSV)

The first line is a header:
userId,movieId,rating,timestamp

_userId_, _movieId_ are integers representing user and movies identifiers correspondingly

_rating_ is a floating point number from 1.0 to 5.0

_timestamp_ is an integer but it won’t be used in the assignment :)

## Reading dataset

Init spark session

In [1]:
from pyspark.sql import SparkSession

spark_session = SparkSession.builder \
    .enableHiveSupport() \
    .appName("recSys")\
    .master("local[8]")\
    .getOrCreate()
sc = spark_session.sparkContext

In [2]:
# You can now use spark session or spark context to read csv.
def skip_header(rdd):
    header = rdd.first()
    rdd = rdd.filter(lambda row: row != header)
    return rdd


In [3]:
from pyspark.mllib import recommendation
from pyspark import SparkContext
def to_rating(ml_latest_row):
    user, movie, rating, timestamp = ml_latest_row.split(',')
    return recommendation.Rating(int(user), int(movie), float(rating))

In [4]:
rdd=sc.textFile("/data/movielens/ratings_100k.csv")
rdd = skip_header(rdd)
rdd = rdd.map(to_rating).repartition(60)

In this assignment you will use collaborative filtering approach for making predictions. To find latent vector representation for users and items we suggest you to use explicit ALS method. Refer to the Spark MLLib documentation for Python API details: https://spark.apache.org/docs/2.1.0/api/python/pyspark.mllib.html

Split dataset into three folds: on each iteration one fold will be used for testing and the other two folds will be used for training. Wrap necessary values with _Rating_ class.

In [5]:
from pyspark.mllib.recommendation import Rating

fold_count = 3
train1, train2,train3 = rdd.randomSplit(weights=[1,1,1], seed=123)
val=[(train1.union(train2),train3),
      (train3.union(train1),train2),
       (train3.union(train2),train1)]
val_cnt=[ (x.count(),y.count()) for x,y in val ]
tot=[x+y for x,y in val_cnt ]


Recommendations: to make your solution more efficient prepare RDDs for all the folds beforehand running the training process.

## Training ALS model

Now it is time to choose training parameters. It is recommended to set the rank equal to 20,
regularization term lambda equal to 0.01 and the number of iterations equal to 5.
Feel free to experiment and choose your own parameters to see how they influence the final score.

In [6]:
rank = 10
iterations =30
lambda_ = 0.2

Train the explicit ALS model on two of three folds and evaluate its performance on remaining test fold. Performance evaluation should be done using the classic RMSE metric.

In [7]:
scores = [0]*3

In [8]:
from pyspark.mllib.recommendation import ALS

In [9]:
def computeRmse(model, data, n):
    """
    Compute RMSE (Root Mean Squared Error).
    """
    from math import sqrt
    from operator import add
    predictions = model.predictAll(data.map(lambda x: (x[0], x[1])))
    predictionsAndRatings = predictions.map(lambda x: ((x[0], x[1]), x[2])) \
      .join(data.map(lambda x: ((x[0], x[1]), x[2]))) \
      .values()
    return sqrt(predictionsAndRatings.map(lambda x: (x[0] - x[1]) ** 2).mean())

In [10]:
fold_count = 3
for test_fold_index in xrange(fold_count):
    als = ALS.train(val[test_fold_index][0],rank,iterations,lambda_)
    scores[test_fold_index]=computeRmse(als,val[test_fold_index][1],val_cnt[test_fold_index][1])

Py4JJavaError: An error occurred while calling o144.trainALSModel.
: org.apache.spark.SparkException: Job 10 cancelled because SparkContext was shut down
	at org.apache.spark.scheduler.DAGScheduler$$anonfun$cleanUpAfterSchedulerStop$1.apply(DAGScheduler.scala:808)
	at org.apache.spark.scheduler.DAGScheduler$$anonfun$cleanUpAfterSchedulerStop$1.apply(DAGScheduler.scala:806)
	at scala.collection.mutable.HashSet.foreach(HashSet.scala:78)
	at org.apache.spark.scheduler.DAGScheduler.cleanUpAfterSchedulerStop(DAGScheduler.scala:806)
	at org.apache.spark.scheduler.DAGSchedulerEventProcessLoop.onStop(DAGScheduler.scala:1668)
	at org.apache.spark.util.EventLoop.stop(EventLoop.scala:83)
	at org.apache.spark.scheduler.DAGScheduler.stop(DAGScheduler.scala:1587)
	at org.apache.spark.SparkContext$$anonfun$stop$8.apply$mcV$sp(SparkContext.scala:1833)
	at org.apache.spark.util.Utils$.tryLogNonFatalError(Utils.scala:1283)
	at org.apache.spark.SparkContext.stop(SparkContext.scala:1832)
	at org.apache.sp

----------------------------------------
Exception happened during processing of request from ('127.0.0.1', 53278)
----------------------------------------


Traceback (most recent call last):
  File "/usr/lib/python2.7/SocketServer.py", line 290, in _handle_request_noblock
    self.process_request(request, client_address)
  File "/usr/lib/python2.7/SocketServer.py", line 318, in process_request
    self.finish_request(request, client_address)
  File "/usr/lib/python2.7/SocketServer.py", line 331, in finish_request
    self.RequestHandlerClass(request, client_address, self)
  File "/usr/lib/python2.7/SocketServer.py", line 652, in __init__
    self.handle()
  File "/usr/local/spark/python/pyspark/accumulators.py", line 235, in handle
    num_updates = read_int(self.rfile)
  File "/usr/local/spark/python/pyspark/serializers.py", line 577, in read_int
    raise EOFError
EOFError


Average RMSE scores on all the three folds and output the result to stdout.
You could refer to publicly available benchmarks (e.g. http://mymedialite.net/examples/datasets.html)
to find out what score to expect.

In [11]:
score = sum(scores) / len(scores)

In [12]:
print(score)

0
